# Dark Skies API

This program pulls weather data associated with a specified latitude, longtiude, and time.

## Basic API Pull

### Import Libraries

In [1]:
import requests
import json
import pandas as pd
import datetime
import numpy as np

### Setup Input Statements

1. Set API key.
2. Latitude, longtidue, datetime.
3. Options, such as exclusions of data.

<b>API Key Notes</b>:<br>
The API key is personal and each user must get their own. This could be an imposed limitation if we allow users to change the underlying crime data. However, if we have the totality of crime data that we need for the analysis, then we can run the API once and store the results on a "database" (terms of service for storing data?).

<b>Input Arguments</b>:<br>
The basic input arguments, besides the API key, are latitude/longtitude (location) and the date and time. I chose to call weather data for 3/2/2019 at 5:30:00 PM.

Notice that the date and time format has "T" in it. This is particular to the API request and the final function that will perform this call will not require the user to acknowledge this.

<b>Parameters to URL</b>:<br>
The request library's <code>params</code> function makes applying the available options to the final URL. I included the <code>exclude</code> option with the arguments <code>minutely, hourly, alerts</code> to keep it manageable for exploratory purposes.

<b>URL Construction</b>:</br>
The final URL is constructed as a concatenation of the base URL provided by the API documentation, the API key, the input arguments, and the parameters.

In [2]:
base_url = 'https://api.darksky.net/forecast/'

api_key = ''
longitude = '38.930267'
latitude = '-77.032745'
datetime = '2019-03-02T17:30:00'

exclude = 'minutely, hourly, alerts'
params = {'exclude': exclude}

url = base_url + api_key + '/' + longitude + ',' + latitude + ',' + datetime

### Request Data from API

Generally, looking for repsonse 200.

In [3]:
response = requests.get(url, params=params)
response

<Response [200]>

### Convert to JSON Object

In [4]:
response_json = json.loads(response.text)
response_json

{'latitude': 38.930267,
 'longitude': -77.032745,
 'timezone': 'America/New_York',
 'currently': {'time': 1551565800,
  'summary': 'Partly Cloudy',
  'icon': 'partly-cloudy-day',
  'precipIntensity': 0,
  'precipProbability': 0,
  'temperature': 44.69,
  'apparentTemperature': 42.57,
  'dewPoint': 34.22,
  'humidity': 0.66,
  'pressure': 1018.63,
  'windSpeed': 4.2,
  'windGust': 6.3,
  'windBearing': 321,
  'cloudCover': 0.52,
  'uvIndex': 0,
  'visibility': 8.39,
  'ozone': 351.31},
 'daily': {'data': [{'time': 1551502800,
    'summary': 'Mostly cloudy throughout the day.',
    'icon': 'partly-cloudy-day',
    'sunriseTime': 1551526834,
    'sunsetTime': 1551567771,
    'moonPhase': 0.88,
    'precipIntensity': 0.0009,
    'precipIntensityMax': 0.0101,
    'precipIntensityMaxTime': 1551502800,
    'precipProbability': 0.74,
    'precipType': 'rain',
    'temperatureHigh': 45.05,
    'temperatureHighTime': 1551564000,
    'temperatureLow': 40.25,
    'temperatureLowTime': 1551618000,


### Convert to Pandas DataFrame Object

Here we can choose to keep certain data pieces to define our <i>instance</i>.

1. Time
2. Latitude
3. Longitude
4. Timezone
5. Temperature (observed)
6. Apparent Temperatures (feels like)
7. Humidity
8. Cloud Cover

In [5]:
time = response_json['currently']['time']
latitude = response_json['latitude']
longitude = response_json['longitude']
timezone = response_json['timezone']
temp_obs = response_json['currently']['temperature']
temp_fl = response_json['currently']['apparentTemperature']
humidity = response_json['currently']['humidity']
cloud_cover = response_json['currently']['cloudCover']

print(time,latitude,longitude,timezone,temp_obs,temp_fl,humidity,cloud_cover)

1551565800 38.930267 -77.032745 America/New_York 44.69 42.57 0.66 0.52


In [6]:
level = pd.DataFrame(data=[[time,latitude,longitude,timezone,temp_obs,temp_fl,humidity,cloud_cover]],
                     columns= ['time','latitude','longitude','timezone','temp_obs','temp_fl','humidity','cloud_cover'])

level

,time,latitude,longitude,timezone,temp_obs,temp_fl,humidity,cloud_cover
0,1551565800,38.930267,-77.032745,America/New_York,44.69,42.57,0.66,0.52


## Calculate Average Temperature

It is likely we will also want the average hourly temperature for the day as well. This could be done before the data is stored in the DataFrame object.

In [7]:
#Construct and Request data just as I did above.
base_url = 'https://api.darksky.net/forecast/'

api_key = ''
longitude = '38.930267'
latitude = '-77.032745'
datetime = '2019-03-02T17:30:00'

exclude = 'minutely, alerts'
params = {'exclude': exclude}

url = base_url + api_key + '/' + longitude + ',' + latitude + ',' + datetime

#Convert to JSON as above.
response = requests.get(url, params=params)
response_json = json.loads(response.text)

#This time though, keep only the hourly data from the response.
hourly_data = response_json['hourly']['data']

In [9]:
#Calculate hourly temp for 3/2/2019.
hourly_temp = []
for index, element in enumerate(hourly_data):
    hourly_temp.append(hourly_data[index]['apparentTemperature'])
    
np.mean(hourly_temp)

36.55083333333334